# Kotlin's collection interfaces

Four important *collection interfaces* are:

- **Tier 1, Iterable**: Provides sequential access to its elements via an `Iterator`.
- **Tier 2, Collection**: Is an iterable that provides additional functionality, allowing to check if the collection contains particular elements.
- **Tier 3, MutableIterable**: Provides a `MutableIterator`, which allows the access and removal of items.
- **Tier 4, MutableCollection**: Provides methods to alter its contained items.   

In [13]:
data class Node<T : Any>(
    var value: T,
    var next: Node<T>? = null
) {
    override fun toString(): String {
        return if (next == null) "$value"
        else "$value -> ${next.toString()}"
    }
}

A mutable collection type is a finite sequence and provides nondestructive sequential access but allows you to modify the collection.

## Making Linked List Iterable

In [14]:
import java.lang.IndexOutOfBoundsException

class LinkedList<T : Any> : Iterable<T> {

    var size = 0
        private set
    private var head: Node<T>? = null
    private var tail: Node<T>? = null

    override fun iterator(): Iterator<T> {
        return LinkedListIterator(this)
    }

    override fun toString(): String {
        return if (isEmpty()) "EMPTY list"
        else head.toString()
    }

    fun isEmpty(): Boolean = size == 0

    fun push(value: T): LinkedList<T> = apply {
        head = Node(value = value, next = head)
        if (null == tail) tail = head
        size++
    }

    fun pop(): T? {
        if (isEmpty()) return null
        val result = head?.value
        head = head?.next
        size--
        if (isEmpty()) tail = null
        return result
    }

    fun removeAfter(node: Node<T>): T? {
        val result = node.next?.value

        if (node.next == tail) {
            tail = node
        }
        if (node.next != null) {
            size--
        }
        node.next = node.next?.next
        return result
    }

    fun nodeAt(index: Int): Node<T>? {
        var currentNode = head
        var currentIndex = 0

        while (currentNode != null && currentIndex < index) {
            currentNode = currentNode.next
            currentIndex++
        }
        return currentNode
    }
    
    // ... Other methods
}

class LinkedListIterator<T : Any>(
    private val list: LinkedList<T>
) : Iterator<T> {

    private var index = 0
    private var lastNode: Node<T>? = null

    override fun hasNext(): Boolean {
        return index < list.size
    }

    override fun next(): T {
        // 1
        if (index >= list.size) throw IndexOutOfBoundsException()
        // 2
        lastNode = if (index == 0) list.nodeAt(0) else lastNode?.next
        // 3
        index++
        return lastNode!!.value
    }

}

1. You check that there are still elements to return. The clients should always check with `hasNext()`method before trying to read a value from the list with `next()`.
2. In the first iteration, there is no `lastNode` set, so you take the first node, after the first iteration, you can get the `next`property of the last node to find the next one.
3. Since the `lastNode` was updated, you need to update the `index` too.

Now, you can iterate your linked list with a regular Kotlin `for` loop:

In [15]:
val list = LinkedList<Int>().push(3).push(2).push(1)
println(list)

for (item in list) {
    println("Double: ${item * 2}")
}

1 -> 2 -> 3
Double: 2
Double: 4
Double: 6


## Becoming a collection

In [16]:
import java.lang.IndexOutOfBoundsException

class LinkedList<T : Any> : Collection<T> {

    override var size = 0
        private set
    private var head: Node<T>? = null
    private var tail: Node<T>? = null

    override fun iterator(): Iterator<T> {
        return LinkedListIterator(this)
    }

    override fun toString(): String {
        return if (isEmpty()) "EMPTY list"
        else head.toString()
    }

    override fun isEmpty(): Boolean = size == 0

    override fun contains(element: T): Boolean {
        for (item in this) {
            if (item == element) return true
        }
        return false
    }

    override fun containsAll(elements: Collection<T>): Boolean {
        for (searched in elements) {
            if (!contains(searched)) return false
        }
        return true
    }

    fun push(value: T): LinkedList<T> = apply {
        head = Node(value = value, next = head)
        if (null == tail) tail = head
        size++
    }

    fun pop(): T? {
        if (isEmpty()) return null
        val result = head?.value
        head = head?.next
        size--
        if (isEmpty()) tail = null
        return result
    }

    fun removeAfter(node: Node<T>): T? {
        val result = node.next?.value

        if (node.next == tail) {
            tail = node
        }
        if (node.next != null) {
            size--
        }
        node.next = node.next?.next
        return result
    }

    fun nodeAt(index: Int): Node<T>? {
        var currentNode = head
        var currentIndex = 0

        while (currentNode != null && currentIndex < index) {
            currentNode = currentNode.next
            currentIndex++
        }
        return currentNode
    }
    
    // ... Other methods
}

class LinkedListIterator<T : Any>(
    private val list: LinkedList<T>
) : Iterator<T> {

    private var index = 0
    private var lastNode: Node<T>? = null

    override fun hasNext(): Boolean {
        return index < list.size
    }

    override fun next(): T {
        if (index >= list.size) throw IndexOutOfBoundsException()
        lastNode = if (index == 0) list.nodeAt(0) else lastNode?.next
        index++
        return lastNode!!.value
    }
}

## Mutable collection

### Mutating while iterating

In [17]:
class LinkedListIterator<T : Any>(
    private val list: LinkedList<T>
) : MutableIterator<T> {
    private var index = 0
    private var lastNode: Node<T>? = null

    override fun hasNext(): Boolean {
        return index < list.size
    }

    override fun next(): T {
        if (index >= list.size) throw IndexOutOfBoundsException()
        lastNode = if (index == 0) list.nodeAt(0) else lastNode?.next
        index++
        return lastNode!!.value
    }

    override fun remove() {
        // 1
        if (index == 1) {
            list.pop()
        } else {
            // 2
            val prevNode = list.nodeAt(index - 2) ?: return
            // 3
            list.removeAfter(prevNode)
            lastNode = prevNode
        }
        index--
    }
}

1. The simplest case is when you're at the beginning of the list. Using `pop` will do the trick.
2. If the iterator is somewhere inside the list, it needs to find the previous node.
3. The iterator need to step back so that `next()`returns the correct method the next time.

## Mutable collection

In [19]:
class LinkedList<T : Any> : MutableCollection<T> {
    override var size = 0
        private set
    private var head: Node<T>? = null
    private var tail: Node<T>? = null

    override fun clear() {
        head = null
        tail = null
        size = 0
    }

    override fun addAll(elements: Collection<T>): Boolean {
        for (element in elements) {
            append(element)
        }
        return true
    }

    override fun add(element: T): Boolean {
        append(element)
        return true
    }

    override fun iterator(): MutableIterator<T> {
        return LinkedListIterator(this)
    }

    override fun retainAll(elements: Collection<T>): Boolean {
        var result = false
        val iterator = this.iterator()
        while (iterator.hasNext()) {
            val item = iterator.next()
            if (!elements.contains(item)) {
                iterator.remove()
                result = true
            }
        }
        return result
    }

    override fun removeAll(elements: Collection<T>): Boolean {
        var result = false
        for (element in elements) {
            result = remove(element) || result
        }
        return result
    }

    override fun remove(element: T): Boolean {
        // 1
        val iterator = iterator()
        // 2
        while (iterator.hasNext()) {
            val item = iterator.next()
            // 3
            if (item == element) {
                iterator.remove()
                return true
            }
        }
        // 4
        return false
    }

    override fun isEmpty(): Boolean = size == 0

    override fun contains(element: T): Boolean {
        for (item in this) {
            if (item == element) return true
        }
        return false
    }

    override fun containsAll(elements: Collection<T>): Boolean {
        for (searched in elements) {
            if (!contains(searched)) return false
        }
        return true
    }

    override fun toString(): String {
        return if (isEmpty()) "EMPTY list"
        else head.toString()
    }

    //...

    fun append(value: T): LinkedList<T> = apply {
        if (isEmpty()) {
            push(value)
            return@apply
        }
        val newNode = Node(value = value)
        tail?.next = newNode
        tail = newNode
        size++
    }
 
    fun push(value: T): LinkedList<T> = apply {
        head = Node(value = value, next = head)
        if (null == tail) tail = head
        size++
    }

    fun pop(): T? {
        if (isEmpty()) return null
        val result = head?.value
        head = head?.next
        size--
        if (isEmpty()) tail = null
        return result
    }

    fun removeAfter(node: Node<T>): T? {
        val result = node.next?.value

        if (node.next == tail) {
            tail = node
        }
        if (node.next != null) {
            size--
        }
        node.next = node.next?.next
        return result
    }

    fun nodeAt(index: Int): Node<T>? {
        var currentNode = head
        var currentIndex = 0

        while (currentNode != null && currentIndex < index) {
            currentNode = currentNode.next
            currentIndex++
        }
        return currentNode
    }

    // ... Other methods
}

class LinkedListIterator<T : Any>(
    private val list: LinkedList<T>
) : MutableIterator<T> {
    private var index = 0
    private var lastNode: Node<T>? = null

    override fun hasNext(): Boolean {
        return index < list.size
    }

    override fun next(): T {
        if (index >= list.size) throw IndexOutOfBoundsException()
        lastNode = if (index == 0) list.nodeAt(0) else lastNode?.next
        index++
        return lastNode!!.value
    }

    override fun remove() {
        if (index == 1) {
            list.pop()
        } else {
            val prevNode = list.nodeAt(index - 2) ?: return
            list.removeAfter(prevNode)
            lastNode = prevNode
        }
        index--
    }
}

1. Get an iterator that will help you iterate through the collection.
2. Create a loop that checks if there are any elements left, and gets the next one
3. Check if the current element is the one you're looking for, and if is, remove it.
4. Return a boolean that signals if an element has been removed.

### Testing Mutable Collection

Removing elements:

In [20]:
val list = LinkedList<Int>()
list.add(3)
list.add(2)
list.add(1)
println(list)

list.remove(2)
println(list)

3 -> 2 -> 1
3 -> 1


Retaining elements:

In [21]:
val list = LinkedList<Int>()
list.add(3)
list.add(2)
list.add(1)
list.add(4)
list.add(5)
println(list)

list.retainAll(listOf(3,4,5))
println(list)

3 -> 2 -> 1 -> 4 -> 5
3 -> 4 -> 5


Remove all elements:

In [22]:
val list = LinkedList<Int>()
list.add(3)
list.add(2)
list.add(1)
list.add(4)
list.add(5)
println(list)

list.removeAll(listOf(3,4,5))
println(list)

3 -> 2 -> 1 -> 4 -> 5
2 -> 1
